In [1]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(0)

In [3]:
def minmax_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [11]:
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i + seq_length, [-1]]  
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

In [12]:
seq_length = 7
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 500

In [13]:
import pandas as pd


In [14]:
xy = np.loadtxt("/content/drive/MyDrive/GOOG.csv", delimiter=",")
xy = xy[::-1]  # reverse order

# split train-test set
train_size = int(len(xy) * 0.7)
train_set = xy[0:train_size]
test_set = xy[train_size - seq_length:]

# scaling data
train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

# make train-test dataset to input
trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

# convert to tensor
trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

[[0.85030035 0.8467458  0.8118273  0.85578526 0.21422344]
 [0.86887666 0.86435311 0.8220704  0.82916419 0.30742724]
 [0.81502236 0.85335746 0.81864915 0.85536811 0.19479387]
 [0.86317179 0.89063932 0.82919776 0.81033564 0.3447918 ]
 [0.88812329 0.89328313 0.8407238  0.82773522 0.61701333]
 [0.86490542 0.89805532 0.88618612 0.90413512 0.29912401]
 [0.87935941 0.86606983 0.84926635 0.83512076 0.10640595]] -> [0.09544568]
[[0.86887666 0.86435311 0.8220704  0.82916419 0.30742724]
 [0.81502236 0.85335746 0.81864915 0.85536811 0.19479387]
 [0.86317179 0.89063932 0.82919776 0.81033564 0.3447918 ]
 [0.88812329 0.89328313 0.8407238  0.82773522 0.61701333]
 [0.86490542 0.89805532 0.88618612 0.90413512 0.29912401]
 [0.87935941 0.86606983 0.84926635 0.83512076 0.10640595]
 [0.90626134 0.90087577 0.90452382 0.91672594 0.09544568]] -> [0.12994561]
[[0.81502236 0.85335746 0.81864915 0.85536811 0.19479387]
 [0.86317179 0.89063932 0.82919776 0.81033564 0.3447918 ]
 [0.88812329 0.89328313 0.8407238  0.8

In [15]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x


net = Net(data_dim, hidden_dim, output_dim, 1)

In [16]:
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [17]:
for i in range(iterations):

    optimizer.zero_grad()
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    loss.backward()
    optimizer.step()
    print(i, loss.item())

0 0.1788242906332016
1 0.13753390312194824
2 0.10344494879245758
3 0.07533486187458038
4 0.05508052557706833
5 0.04559266194701195
6 0.047641731798648834
7 0.05311890318989754
8 0.053760088980197906
9 0.04969247058033943
10 0.043973930180072784
11 0.039057981222867966
12 0.036046020686626434
13 0.03496777266263962
14 0.03526013717055321
15 0.03615379333496094
16 0.03694498538970947
17 0.037178266793489456
18 0.03671411797404289
19 0.03567424416542053
20 0.03432890772819519
21 0.032986920326948166
22 0.031909480690956116
23 0.031250961124897
24 0.031030094251036644
25 0.031136661767959595
26 0.03137471154332161
27 0.03153301402926445
28 0.03145994246006012
29 0.031112806871533394
30 0.030561113730072975
31 0.029946375638246536
32 0.0294199138879776
33 0.029085209593176842
34 0.02896459773182869
35 0.02899925783276558
36 0.02908031828701496
37 0.029098033905029297
38 0.028988448902964592
39 0.02875727042555809
40 0.028471650555729866
41 0.028224924579262733
42 0.028090177103877068
43 0.0